# Solution Assignment 2: MNIST

This is the solution for the second assignment of the Implementing Artificial Neural Networks with TensorFlow course. Please find the according assignment sheet on studip.

## Imports
We are going to use `tensorflow` to implement the neural network, `numpy` for the preparation of our training data and `matplotlib` to plot the training process of the network and the change of weights over time. The `struct` library is required to read the header of the binary MNIST data files and `os` is required to join directory paths. Finally, we activate inline plotting by calling `%matplotlib notebook`.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import struct
import os

%matplotlib notebook

## The MNIST helper class
When an instance of this class is created, the training and test images and coresponding labels are read from the MNIST binary files from `directory`. Further, 10% of the training data is separated into a validation data-set.

The class provides three methods to iterate over the training, validation and test data respectively. If `batch_size` is set to a value `<= 0`, the method returns all data and labels at once.

In [2]:
class MNIST():
    def __init__(self, directory):
        self._directory = directory
        
        self._training_data = self._load_binaries("train-images.idx3-ubyte")
        self._training_labels = self._load_binaries("train-labels.idx1-ubyte")
        self._test_data = self._load_binaries("t10k-images.idx3-ubyte")
        self._test_labels = self._load_binaries("t10k-labels.idx1-ubyte")
        
        np.random.seed(0)
        samples_n = self._training_labels.shape[0]
        random_indices = np.random.choice(samples_n, samples_n // 10, replace = False)
        np.random.seed()
        
        self._validation_data = self._training_data[random_indices]
        self._validation_labels = self._training_labels[random_indices]
        self._training_data = np.delete(self._training_data, random_indices, axis = 0)
        self._training_labels = np.delete(self._training_labels, random_indices)
    
    def _load_binaries(self, file_name):
        path = os.path.join(self._directory, file_name)
        
        with open(path, 'rb') as fd:
            check, items_n = struct.unpack(">ii", fd.read(8))

            if "images" in file_name and check == 2051:
                height, width = struct.unpack(">II", fd.read(8))
                images = np.fromfile(fd, dtype = 'uint8')
                return np.reshape(images, (items_n, height, width))
            elif "labels" in file_name and check == 2049:
                return np.fromfile(fd, dtype = 'uint8')
            else:
                raise ValueError("Not a MNIST file: " + path)
    
    
    def get_training_batch(self, batch_size):
        return self._get_batch(self._training_data, self._training_labels, batch_size)
    
    def get_validation_batch(self, batch_size):
        return self._get_batch(self._validation_data, self._validation_labels, batch_size)
    
    def get_test_batch(self, batch_size):
        return self._get_batch(self._test_data, self._test_labels, batch_size)
    
    def _get_batch(self, data, labels, batch_size):
        samples_n = labels.shape[0]
        if batch_size <= 0:
            batch_size = samples_n
        
        random_indices = np.random.choice(samples_n, samples_n, replace = False)
        data = data[random_indices]
        labels = labels[random_indices]
        for i in range(samples_n // batch_size):
            on = i * batch_size
            off = on + batch_size
            yield data[on:off], labels[on:off]
    
    
    def get_sizes(self):
        training_samples_n = self._training_labels.shape[0]
        validation_samples_n = self._validation_labels.shape[0]
        test_samples_n = self._test_labels.shape[0]
        return training_samples_n, validation_samples_n, test_samples_n
    
        
    

## Creating a new MNIST instance
Since the MNIST class reads the MNIST data every time we create a new instance, we are going to create a single instance here and reuse it in the cells bellow.

This script assumes, that the MNIST data files are located in a subdirectory "data".

In [3]:
mnist = MNIST("./data/")

## Investigating the data
Before we start to use the data we should always try to understand its structure and double check, that data and labels are assigned correctly. Therefore, we plot some samples and the respective labels.

In [4]:
images, labels = next(mnist.get_training_batch(25))

fig, axs = plt.subplots(3, 5)
for i, ax in enumerate(np.reshape(axs, [-1])):
    ax.matshow(images[i], cmap = 'gray')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_title(labels[i])

<IPython.core.display.Javascript object>